# Dashboard 1: Ski Resort Slope Density

Build an Application containing a density mapbox graph that displays color intensity based on `Total slopes`. This map should be centered roughly on the US/Canada.

* Add a slider to this application that filters the resorts mapped to those with a Price below the value selected in the slider.
* Add a radio button that toggles between resorts with night skiing and without night skiing. 
* Consider making your title dynamic based on the price input

In [1]:
import numpy as np
import pandas as pd

from jupyter_dash import JupyterDash
from dash import dcc, html
from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate

import plotly.express as px

In [11]:
resorts = pd.read_csv("Ski Resorts/resorts.csv", encoding = "ISO-8859-1").query("Country in ['United States', 'Canada']")
resorts.head()

,ID,Resort,Latitude,Longitude,Country,Continent,Price,Season,Highest point,Lowest point,...,Snow cannons,Surface lifts,Chair lifts,Gondola lifts,Total lifts,Lift capacity,Child friendly,Snowparks,Nightskiing,Summer skiing
3,4,Red Mountain Resort-Rossland,49.105520,-117.846280,Canada,North America,60,December - April,2075,1185,...,0,2,5,1,8,9200,Yes,Yes,Yes,No
10,11,Fernie,49.504175,-115.062867,Canada,North America,67,December - April,2134,1052,...,11,3,7,0,10,14514,Yes,Yes,No,No
11,12,Sun Peaks,50.884468,-119.882329,Canada,North America,62,November - April,2082,1198,...,0,6,6,0,12,13895,Yes,Yes,Yes,No
12,13,Panorama,50.736999,-119.120561,Canada,North America,62,December - April,2365,1140,...,0,3,6,4,13,11890,Yes,Yes,Yes,No
21,22,Steamboat,35.754022,-109.853751,United States,North America,120,November - April,3221,2103,...,0,1,14,2,17,32720,Yes,Yes,Yes,No


In [17]:
#int app 
app = JupyterDash(__name__)

#front-end
app.layout = html.Div([
    html.H1(id="title"),
    dcc.Slider(id="price-slider", min=0, max=150, step=10, value=150),
    dcc.RadioItems(
        id="night-ski-toggle", options=[
            {"label":" Has night Skiing", "value": "Yes"},
            {"label": "No Night Skiing", "value":"No"}],
        value="No"
        ),
    dcc.Graph(id="resort-map")
    ])

#back-end
@app.callback(
    Output("title", "children"),
    Output("resort-map", "figure"),
    Input("price-slider", "value"),
    Input("night-ski-toggle", "value")
)

def snow_map(price, night_ski):
    title = f"Resort with a ticket price less than ${price}."
    
    df = resorts.loc[(resorts["Price"] < price) & resorts["Nightskiing"] == night_ski]
    
    fig = px.density_mapbox(
            df,
            lat="Latitude",
            lon="Longitude",
            z="Total slopes",
            hover_name="Resort",
            center={"lat": 45, "lon": -100},
            zoom=2.5,
            mapbox_style="stamen_terrain",
            color_continuous_scale="blues",
            width=1000,
            height=800
    )
    
    return title, fig

#display
app.run_server(mode="inline",debug=True, port=8432)

# Dashboard 2: Top Resorts by Selected Metric

Build an Application that outputs a bar chart of the top 10 resorts based on the dataframe column users select in a dropdown menu. 

* Add a checklist to allow users to select whether they want to limit results to Canada, the US, or include both countries. 
* Add a radio button that toggles between resorts with night skiing and without night skiing. 
* Add an interactive element that allows users to select whether they want to sort the plotted dataframe in ascending or descending order.


In [15]:
from jupyter_dash import JupyterDash
from dash import dcc, html
from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate

import plotly.express as px
import pandas as pd

resorts = (
    pd.read_csv("Ski Resorts/resorts.csv", encoding = "ISO-8859-1")
    .query("Country in ['United States', 'Canada']")
)

app = JupyterDash(__name__)

app.layout = html.Div([
    html.H1(id="title"),
    dcc.Checklist(
        id="country-checklist", 
        options=["Canada", "United States"],
        value=["Canada", "United States"],
    ),
    dcc.Dropdown(
        id="column-picker", 
        options=resorts.select_dtypes("number").columns[3:],
        value="Price"
 ),
    dcc.RadioItems(
        id="ascending-selector", 
        options=[
            {"label": "Ascending", "value": True},
            {"label": "Descending", "value": False}],
        value=False,
    ),
    dcc.Graph(id="metric-bar"),
], style={"width":1200})


@app.callback(
    Output("title", "children"),
    Output("metric-bar", "figure"),
    Input("country-checklist", "value"),
    Input("column-picker", "value"),
    Input("ascending-selector", "value")
)

def snow_map(country, column, ascending):
    
    title = f"Top 10 resorts by {column}"  
    
    df = resorts.query("Country in @country")
    
    fig = px.bar(
        df.sort_values(column, ascending=ascending).iloc[:10],
        x="Resort",
        y=column,
#         color="Country",
        width=1000,
        height=800
    )

    return title, fig

app.run_server(debug=True, mode="inline", port=8666)